# Demo-1+: Vergleich von Zeitreihen mit Liniendiagrammen 

## Importing Dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wgts
from IPython.display import display
import numpy as np

## Loading Datasets

In [2]:
covid_19_dataset = pd.read_csv('cleaned_dataset.csv')
regions_dataset = pd.read_csv('regions_dataset.csv')

## Plotting Functions

### Region Based Plotting

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_region_timeseries(region: str, from_date: str, to_date: str):
    """
    Plots a time series graph for a given country, continent, or the world within a specified date range.

    Parameters:
        region (str): The country, continent, or "World".
        from_date (str): Start date in 'YYYY-MM-DD' format.
        to_date (str): End date in 'YYYY-MM-DD' format.
    """
    global regions_dataset  # Assumes regions_dataset is defined somewhere in the notebook
    df = pd.read_csv("cleaned_dataset.csv")

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter dataset for the given date range
    df = df[(df['Date'] >= from_date) & (df['Date'] <= to_date)]

    if region == "World":
        # Aggregate data for the entire world
        df_region = df.groupby("Date")[["Confirmed", "Recovered", "Deaths"]].sum().reset_index()
    elif region in regions_dataset["Continent"].unique():
        # Get countries belonging to the specified continent
        countries_in_continent = regions_dataset[regions_dataset["Continent"] == region]["Country"].unique()
        df_region = df[df["Country"].isin(countries_in_continent)].groupby("Date")[["Confirmed", "Recovered", "Deaths"]].sum().reset_index()
    else:
        # Treat as a country
        df_region = df[df["Country"] == region].sort_values(by="Date")

    # Compute cumulative sum
    df_region["Total Confirmed"] = df_region["Confirmed"].cumsum()
    df_region["Total Recovered"] = df_region["Recovered"].cumsum()
    df_region["Total Deaths"] = df_region["Deaths"].cumsum()

    # Plot time series graph
    plt.figure(figsize=(12, 6))
    plt.plot(df_region["Date"], df_region["Total Confirmed"], label="Confirmed Cases", color='blue')
    plt.plot(df_region["Date"], df_region["Total Recovered"], label="Recovered Cases", color='green')
    plt.plot(df_region["Date"], df_region["Total Deaths"], label="Deaths", color='red')

    # Formatting the plot
    plt.xlabel("Date")
    plt.ylabel("Cumulative Count")
    plt.title(f"COVID-19 Time Series for {region} ({from_date} to {to_date})")
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)

    # Show plot
    plt.show()

### Multiple Time Series Plotting

In [5]:
def plot_multiple_timeseries(regions: list, column: str, from_date: str, to_date: str):
    """
    Plots a time series graph for multiple regions (countries, continents, or World) for a given column.

    Parameters:
        regions (list): List of regions (country names, continent names, or "World").
        column (str): The column to plot ("Confirmed", "Recovered", or "Deaths").
        from_date (str): Start date in 'YYYY-MM-DD' format.
        to_date (str): End date in 'YYYY-MM-DD' format.
    """

    df = pd.DataFrame(covid_19_dataset)

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter dataset for date range
    df = df[(df['Date'] >= from_date) & (df['Date'] <= to_date)]

    # Initialize the plot
    plt.figure(figsize=(12, 6))

    for region in regions:
        if region == "World":
            # Aggregate data globally
            df_region = df.groupby("Date")[column].sum().reset_index()
        elif region in regions_dataset["Continent"].unique():
            # Get countries belonging to the continent
            countries_in_continent = regions_dataset[regions_dataset["Continent"] == region]["Country"].unique()
            df_region = df[df["Country"].isin(countries_in_continent)].groupby("Date")[column].sum().reset_index()
        else:
            # Treat as a country
            df_region = df[df["Country"] == region].sort_values(by="Date")
    
        # Compute cumulative sum for the selected column
        df_region[f"Total {column}"] = df_region[column].cumsum()

        # Plot the time series
        plt.plot(df_region["Date"], df_region[f"Total {column}"], label=f"{region} ({column})")

    # Formatting the plot
    plt.xlabel("Date")
    plt.ylabel("Cumulative Count")
    plt.title(f"COVID-19 {column} Cases Over Time ({from_date} to {to_date})")
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True)

    # Show plot
    plt.show()

## Creating Widgets

### Creating Countries & Continents Dropdown Menus

In [6]:
continent_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Continent'].unique(),
    description='Continent:',
    disabled=False,
)

continent_filter = regions_dataset['Continent'] == continent_dropdown_menu.value
countries = regions_dataset[continent_filter]
countries = countries['Country']

countries_dropdown_menu = wgts.Dropdown(
    options= countries,
    description='Country:',
    disabled=False,
)

### Creating From/To Day DatePickers

In [7]:
import datetime
# minimum date value in dataset
dateMin = datetime.datetime.strptime(covid_19_dataset['Date'].min(), '%Y-%m-%d').date()
# maximum date value in dataset
dateMax = datetime.datetime.strptime(covid_19_dataset['Date'].max(), '%Y-%m-%d').date()

from_datepicker = wgts.DatePicker(
    description='From:',
    disabled=False,
    value=dateMin,
    max=dateMax,
    min=dateMin
)

to_datepicker = wgts.DatePicker(
    description='To:',
    disabled=False,
    value=dateMin,
    max=dateMax,
    min=dateMin
)


### Creating Reload, Remove, Add Buttons

In [ ]:
reload_button = wgts.Button(
    description='Reload',
    disabled=False,
    button_style='info',
    tooltip='Reload Graph',
    icon='check'
)

remove_button = wgts.Button(
    description='Remove',
    disabled=False,
    button_style='danger',
    tooltip='Remove Region',
    icon='minus'
)

add_button = wgts.Button(
    description='Add',
    disabled=False,
    button_style='success',
    tooltip='Add Region',
    icon='plus'
)

### Region Selection Menu

In [ ]:
selected_regions_menu = wgts.Select(
    description='Regions:',
    disabled=False
)

### Column Selection Toggle Buttons

In [ ]:
column_selection_buttons = wgts.ToggleButtons(
    options=['Confirmed', 'Recovered', 'Deaths'],
    description='Column:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Confirmed Cases Of Covid-19', 'Recoveries From Covid-19', 'Deaths From Covid-19'],
)